# Events

In [15]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [36]:
import polars as pl
from discontinuitypy.datasets import IDsDataset
from beforerr.polars import pl_norm

from sunpy.time import TimeRange
from datetime import timedelta
from space_analysis.utils.speasy import Variables

from bokeh.io import output_notebook
output_notebook()

Loading BokehJS ...

In [17]:
import astropy.units as u
from astropy.constants import m_p

def thermal_spd2temp(speed, speed_unit=u.km / u.s):
    return (m_p * (speed * speed_unit) ** 2 / 2).to("eV").value


def df_thermal_spd2temp(ldf: pl.LazyFrame, speed_col, speed_unit=u.km / u.s):
    df = ldf.collect()
    return df.with_columns(
        plasma_temperature=thermal_spd2temp(df[speed_col].to_numpy(), speed_unit)
    ).lazy()

In [18]:
def get_and_process_data(
    mag_dataset, mag_parameters, plasma_dataset, plasma_parameters, timerange, tau, ts
):
    # define variables
    mag_vars = Variables(
        dataset=mag_dataset,
        parameters=mag_parameters,
        timerange=timerange,
    ).retrieve_data()

    plasma_vars = Variables(
        dataset=plasma_dataset,
        parameters=plasma_parameters,
        timerange=timerange,
    ).retrieve_data()

    # get column names
    bcols = mag_vars.data[0].columns
    density_col = plasma_vars.data[0].columns[0]
    vec_cols = plasma_vars.data[1].columns
    temperature_col = plasma_vars.data[2].columns[0]

    # get data
    mag_data = mag_vars.to_polars()
    plasma_data = (
        plasma_vars.to_polars()
        .with_columns(plasma_speed=pl_norm(vec_cols))
        .rename({density_col: "plasma_density"})
    )
    # process temperature data
    if plasma_vars.data[2].unit == "km/s":
        plasma_data = plasma_data.pipe(df_thermal_spd2temp, temperature_col)
    else:
        plasma_data = plasma_data.rename({temperature_col: "plasma_temperature"})

    return IDsDataset(
        mag_data=mag_data,
        plasma_data=plasma_data,
        tau=tau,
        ts=ts,
        bcols=bcols,
        vec_cols=vec_cols,
        density_col="plasma_density",
        speed_col="plasma_speed",
        temperature_col="plasma_temperature",
    )

In [19]:
start = "2019-04-07T01:00"
end = "2019-04-07T12:00"

earth_start = "2019-04-09"
earth_end = "2019-04-12"

In [20]:
psp_timerange = TimeRange(start, end)
earth_timerange = TimeRange(earth_start, earth_end)

In [21]:
def validate(timerange):
    if isinstance(timerange, TimeRange):
        return [timerange.start.to_string(), timerange.end.to_string()]

In [28]:
tau = timedelta(seconds=16)
ts = timedelta(seconds=1 / 180)

mag_dataset = "PSP_FLD_L2_MAG_RTN"
mag_parameters = ["psp_fld_l2_mag_RTN"]

# plasma_dataset = "PSP_SWP_SPI_SF00_L3_MOM"
# plasma_parameters = ["DENS", "VEL_RTN_SUN", "TEMP", "SUN_DIST"]
# fname = "../data/psp_ids_dataset.arrow"

plasma_dataset = "PSP_SWP_SPC_L3I"
plasma_parameters = ["np_moment_gd", "vp_moment_RTN_gd", "wp_moment_gd"]


timerange = validate(psp_timerange)

fname = "../data/psp_ids_dataset_spc.arrow"

In [31]:
psp_ids_dataset = get_and_process_data(
    mag_dataset, mag_parameters, plasma_dataset, plasma_parameters, timerange, tau, ts
).find_events().update_candidates_with_plasma_data().export(fname)

10-Feb-24 17:58:50: UserWarning: Distributing <class 'pandas.core.frame.DataFrame'> object. This may take some time.



Distributing Dataframe:   0%           Elapsed time: 00:00, estimated remaining time: ?

Estimated completion of line 3:   0%           Elapsed time: 00:00, estimated remaining time: ?

Loading BokehJS ...

In [33]:
psp_ids_dataset.plot(offset=timedelta(seconds=5))

2024-02-10 17:59:20.159 | INFO     | discontinuitypy.datasets:overview_plot:172 - Overview plot: 2019-04-07 07:20:56 - 2019-04-07 07:21:12
2024-02-10 17:59:20.159 | DEBUG    | discontinuitypy.datasets:log_event_change:79 - CHANGE INFO
        n.change: -15.301864624023438
        v.ion.change: -11.389994719711808
        T.change: -15.35660171508789
        v.Alfven.change: 11.979870753367976
        v.ion.change.l: 88.78097595793847
        v.Alfven.change.l: 133.87318099874705
        


:Layout
   .Overlay.I   :Overlay
      .NdOverlay.I :NdOverlay   [B comp]
         :Curve   [time]   (B)
      .VLine.I     :VLine   [x,y]
      .VLine.II    :VLine   [x,y]
   .Overlay.II  :Overlay
      .Curve.I   :Curve   [time]   (plasma_density)
      .Scatter.I :Scatter   [time]   (plasma_density)
      .VLine.I   :VLine   [x,y]
      .VLine.II  :VLine   [x,y]
   .Overlay.III :Overlay
      .NdOverlay.I :NdOverlay   [veloity comp]
         :Curve   [time]   (v)
      .VLine.I     :VLine   [x,y]
      .VLine.II    :VLine   [x,y]
   .Overlay.IV  :Overlay
      .Curve.I  :Curve   [time]   (plasma_temperature)
      .VLine.I  :VLine   [x,y]
      .VLine.II :VLine   [x,y]

In [10]:
from discontinuitypy.utils.basic import resample

ts = timedelta(seconds=1 / 11)
fname = "../data/psp_ids_dataset_r.arrow"

psp_ids_dataset_r = psp_ids_dataset.model_copy(deep=True)
psp_ids_dataset_r.data = resample(psp_ids_dataset_r.data, ts)
psp_ids_dataset_r.ts = ts
psp_ids_dataset_r.find_events().update_candidates_with_plasma_data().export(fname)

09-Feb-24 17:12:25: UserWarning: Distributing <class 'pandas.core.frame.DataFrame'> object. This may take some time.



Distributing Dataframe:   0%           Elapsed time: 00:00, estimated remaining time: ?

(_deploy_ray_func pid=60598) UserWarning: Traceback (most recent call last):
(_deploy_ray_func pid=60598)   File "/Users/zijin/micromamba/envs/psp_conjunction/lib/python3.11/site-packages/pdpipe/__init__.py", line 85, in <module>
(_deploy_ray_func pid=60598)     from . import skintegrate
(_deploy_ray_func pid=60598)   File "/Users/zijin/micromamba/envs/psp_conjunction/lib/python3.11/site-packages/pdpipe/skintegrate.py", line 20, in <module>
(_deploy_ray_func pid=60598)     from sklearn.base import BaseEstimator
(_deploy_ray_func pid=60598) ModuleNotFoundError: No module named 'sklearn'
(_deploy_ray_func pid=60598) 
(_deploy_ray_func pid=60598) UserWarning: pdpipe: Scikit-learn or skutil import failed. Scikit-learn-dependent pipeline stages will not be loaded.
(_deploy_ray_func pid=60598) UserWarning: Traceback (most recent call last):
(_deploy_ray_func pid=60598)   File "/Users/zijin/micromamba/envs/psp_conjunction/lib/python3.11/site-packages/pdpipe/__init__.py", line 105, in <module>

Estimated completion of line 9:   0%           Elapsed time: 00:00, estimated remaining time: ?

IDsDataset(name=None, events=shape: (89, 105)
┌────────────┬───────┬───────────┬───────────┬───┬────────────┬───────────┬────────────┬───────────┐
│ time       ┆ count ┆ B_std     ┆ B_mean    ┆ … ┆ T.change   ┆ B.change  ┆ v.Alfven.c ┆ v.Alfven. │
│ ---        ┆ ---   ┆ ---       ┆ ---       ┆   ┆ ---        ┆ ---       ┆ hange      ┆ change.l  │
│ datetime[n ┆ u32   ┆ f64       ┆ f64       ┆   ┆ f64        ┆ f64       ┆ ---        ┆ ---       │
│ s]         ┆       ┆           ┆           ┆   ┆            ┆           ┆ f64        ┆ f64       │
╞════════════╪═══════╪═══════════╪═══════════╪═══╪════════════╪═══════════╪════════════╪═══════════╡
│ 2019-04-07 ┆ 176   ┆ 42.543101 ┆ 85.393798 ┆ … ┆ -20.301338 ┆ 0.083642  ┆ -141.27939 ┆ 317.01786 │
│ 01:05:28   ┆       ┆           ┆           ┆   ┆            ┆           ┆ 1          ┆ 6         │
│ 2019-04-07 ┆ 176   ┆ 37.298421 ┆ 85.808691 ┆ … ┆ 19.339516  ┆ 2.537321  ┆ 157.987252 ┆ -343.5237 │
│ 01:06:08   ┆       ┆           ┆           

## Wind dataset


In [10]:
ts = timedelta(seconds=1 / 11)
tau = timedelta(seconds=30)

mag_dataset = "WI_H2_MFI"
# mag_parameters = ["BGSM"]
mag_parameters = ["BGSE"]

# plasma_dataset="WI_K0_SWE"
# plasma_parameters=["Np", "V_GSM", "THERMAL_SPD"]

plasma_dataset="WI_PM_3DP"
plasma_parameters=["P_DENS", "P_VELS", "P_TEMP"] # Proton velocity vector (GSE)

timerange = validate(earth_timerange)

fname = "../data/wind_ids_dataset_r.arrow"

In [11]:
wind_ids_dataset = get_and_process_data(
    mag_dataset, mag_parameters, plasma_dataset, plasma_parameters, timerange, tau, ts
).find_events().update_candidates_with_plasma_data().export(fname)

09-Feb-24 16:50:28: UserWarning: Distributing <class 'pandas.core.frame.DataFrame'> object. This may take some time.



Distributing Dataframe:   0%           Elapsed time: 00:00, estimated remaining time: ?

(_deploy_ray_func pid=58910) UserWarning: Distributing <class 'dict'> object. This may take some time.
Distributing Dataframe: 100%██████████ Elapsed time: 00:00, estimated remaining time: 00:00
(_deploy_ray_func pid=58908) UserWarning: Traceback (most recent call last):
(_deploy_ray_func pid=58908)   File "/Users/zijin/micromamba/envs/psp_conjunction/lib/python3.11/site-packages/pdpipe/__init__.py", line 85, in <module>
(_deploy_ray_func pid=58908)     from . import skintegrate
(_deploy_ray_func pid=58908)   File "/Users/zijin/micromamba/envs/psp_conjunction/lib/python3.11/site-packages/pdpipe/skintegrate.py", line 20, in <module>
(_deploy_ray_func pid=58908)     from sklearn.base import BaseEstimator
(_deploy_ray_func pid=58908) ModuleNotFoundError: No module named 'sklearn'
(_deploy_ray_func pid=58908) 
(_deploy_ray_func pid=58908) UserWarning: pdpipe: Scikit-learn or skutil import failed. Scikit-learn-dependent pipeline stages will not be loaded.
(_deploy_ray_func pid=58908) UserWa

Estimated completion of line 3:   0%           Elapsed time: 00:00, estimated remaining time: ?

In [12]:
import hvplot.polars

In [16]:
wind_ids_dataset.plot()

2024-02-09 16:50:46.654 | INFO     | discontinuitypy.datasets:overview_plot:167 - Overview plot: 2019-04-11 16:45:00 - 2019-04-11 16:45:30
2024-02-09 16:50:46.654 | DEBUG    | discontinuitypy.datasets:log_event_change:79 - CHANGE INFO
        n.change: -0.0016632080078125
        v.ion.change: -0.07276110613753417
        T.change: 0.10945892333984375
        v.Alfven.change: 0.16010592925675837
        v.ion.change.l: -7.858156252181972
        v.Alfven.change.l: -15.310337071702092
        


:Layout
   .Overlay.I   :Overlay
      .NdOverlay.I :NdOverlay   [B comp]
         :Curve   [time]   (B)
      .VLine.I     :VLine   [x,y]
      .VLine.II    :VLine   [x,y]
   .Overlay.II  :Overlay
      .Curve.I   :Curve   [time]   (plasma_density)
      .Scatter.I :Scatter   [time]   (plasma_density)
      .VLine.I   :VLine   [x,y]
      .VLine.II  :VLine   [x,y]
   .Overlay.III :Overlay
      .NdOverlay.I :NdOverlay   [veloity comp]
         :Curve   [time]   (v)
      .VLine.I     :VLine   [x,y]
      .VLine.II    :VLine   [x,y]
   .Overlay.IV  :Overlay
      .Curve.I  :Curve   [time]   (plasma_temperature)
      .VLine.I  :VLine   [x,y]
      .VLine.II :VLine   [x,y]